In [2]:
%load_ext autoreload
%autoreload 2

In [14]:
import json
import os

PREFIX = 'gptoss20-gpt'
DATASET_FILE_PATH = f"evaluation/results/{PREFIX}_dataset.json"

if os.path.exists(DATASET_FILE_PATH):
    with open(DATASET_FILE_PATH, 'r') as f:
        dataset = json.load(f)

In [15]:
print(dataset[0])
dataset[0].pop('annotated_accuracy')
print(dataset[0])

{'prompt': 'Return the 12 months of 2023 with total revenue and total units sold; columns: month_start, total_revenue, total_units; order by month_start ASC.', 'gt_sql': "SELECT DATE_TRUNC('month', Sold_Date) AS month_start, SUM(Total_Sale_Value) AS total_revenue, SUM(Qty_Sold) AS total_units FROM sales WHERE CAST(Sold_Date AS VARCHAR) LIKE '2023%' GROUP BY DATE_TRUNC('month', Sold_Date) ORDER BY month_start ASC", 'gt_data': '   month_start  total_revenue  total_units\n0   2023-01-01  434172.808300      33653.0\n1   2023-02-01  411762.718244      31545.0\n2   2023-03-01  423070.838309      32460.0\n3   2023-04-01  384479.678311      30199.0\n4   2023-05-01  381326.428023      31154.0\n5   2023-06-01  364806.737856      29082.0\n6   2023-07-01  353284.207838      27932.0\n7   2023-08-01  401061.537729      31903.0\n8   2023-09-01  422554.967469      31847.0\n9   2023-10-01  454231.057104      38208.0\n10  2023-11-01  511104.287418      41043.0\n11  2023-12-01  874263.668331      60941.0

In [ ]:
import os
import json
from Agent.utils import *
from Agent.data_agent import SalesDataAgent

<module 'langgraph.version' from '/Users/simop/miniconda3/lib/python3.11/site-packages/langgraph/version.py'>


In [ ]:
# Optional: ensure deps (uncomment if needed)
# %pip install -q langgraph langchain-ollama duckdb pandas pyarrow matplotlib langchain-ollama

agent = SalesDataAgent(
    enable_tracing=True,
    phoenix_endpoint="http://localhost:6006/v1/traces",
    project_name="evaluating-agent",
    model="llama3.2:3b"
)
#ret = agent.run("What was the most popular product SKU?")
#print(ret.get("answer"))


Overriding of current TracerProvider is not allowed
Attempting to instrument while already instrumented


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: evaluating-agent
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://localhost:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [8]:
agent.check_model()
agent.draw_graph()

Server is running locally


KeyboardInterrupt: 

In [ ]:
PREFIX = 'gpt'
DATASET_FILE_PATH = f"evaluation/{PREFIX}_dataset.json"
if os.path.exists(DATASET_FILE_PATH):
    with open(DATASET_FILE_PATH, 'r') as f:
        dataset = json.load(f)

for i in range(len(dataset)):
    if (dataset[i].get('gen_sql') and dataset[i].get('gen_data')):
        print(f'\nSkipping question {i+1}/{len(dataset)} as generated results were already added.')
        continue
    print(f"------- Question {i+1}/{len(dataset)} -------")

    ret = agent.run(dataset[i]['prompt'], only_lookup=True)
    dataset[i]['gen_sql']=ret['sql_query']
    dataset[i]['gen_data']=ret['data']
    csv_path = f"evaluation/csv_queries/{PREFIX}_{i}_gen.csv"
    result_rows = text_to_csv(ret['data'])
    save_csv(result_rows, csv_path)
    dataset[i]["gen_csv_path"]=csv_path

with open(DATASET_FILE_PATH, 'w') as f:
    json.dump(dataset, f, indent=2)


Skipping question 1/21 as generated results were already added.

Skipping question 2/21 as generated results were already added.

Skipping question 3/21 as generated results were already added.

Skipping question 4/21 as generated results were already added.

Skipping question 5/21 as generated results were already added.

Skipping question 6/21 as generated results were already added.

Skipping question 7/21 as generated results were already added.

Skipping question 8/21 as generated results were already added.

Skipping question 9/21 as generated results were already added.
------- Question 10/21 -------
[Agent] Running only lookup_sales_data
Generated SQL Query:
 SELECT 
    MONTH(Sold_Date) AS Month,
    SUM(Total_Sale_Value) AS Total_Revenue,
    SUM(Qty_Sold) AS Total_Units_Sold
FROM sales
WHERE CAST(Sold_Date AS VARCHAR) LIKE '%2023-%'
GROUP BY MONTH(Sold_Date)
ORDER BY MONTH(Sold_Date) ASC
------- Question 11/21 -------
[Agent] Running only lookup_sales_data
Generated SQL Que

In [ ]:
PREFIX = 'claude'
DATASET_FILE_PATH = f"evaluation/{PREFIX}_dataset.json"
if os.path.exists(DATASET_FILE_PATH):
    with open(DATASET_FILE_PATH, 'r') as f:
        dataset = json.load(f)
i=9
data = dataset[i]
ret = best_of_n(
    agent, 
    data['prompt'],
    expected_csv=data['gt_csv_path'],
    csv_path=data['gen_csv_path'],
    n=3,
    temperature=[0.05,0.15] #Or also single values: 0.1
)


--- Attempt 1/3 (temperature=0.05) ---
[Agent] Running only lookup_sales_data
Generated SQL Query:
 SELECT COUNT(*) ,  MONTH(CAST(Sold_Date AS VARCHAR)) ,  YEAR(CAST(Sold_Date AS VARCHAR)) FROM sales WHERE Sold_Date LIKE '%2023%' GROUP BY MONTH(CAST(Sold_Date AS VARCHAR)), YEAR(CAST(Sold_Date AS VARCHAR))
Error accessing data: Binder Error: No function matches the given name and argument types '~~(DATE, STRING_LITERAL)'. You might need to add explicit type casts.
	Candidate functions:
	~~(VARCHAR, VARCHAR) -> BOOLEAN

Error in attempt 1: No columns to parse from file

--- Attempt 2/3 (temperature=0.10) ---
[Agent] Running only lookup_sales_data
Generated SQL Query:
 SELECT COUNT(Total_Sale_Value), MONTH(CAST(Sold_Date AS VARCHAR)) 
FROM sales 
WHERE Sold_Date LIKE '%2023%' GROUP BY MONTH(CAST(Sold_Date AS VARCHAR));
Error accessing data: Binder Error: No function matches the given name and argument types '~~(DATE, STRING_LITERAL)'. You might need to add explicit type casts.
	Candidate